In [ ]:
from net2brain.utils.download_datasets import DatasetNSD_872
from net2brain.feature_extraction import FeatureExtractor
from net2brain.evaluations.ridge_regression import Ridge_Encoding
from net2brain.evaluations.plotting import Plotting
import os

In [ ]:
paths_NSD_872 = DatasetNSD_872().load_dataset()

In [ ]:
stimuli_path = paths_NSD_872["NSD_872_images"]
roi_path = paths_NSD_872["NSD_872_fmri"] 

# join roi path with the prf-visualrois folder
roi_path = os.path.join(roi_path, "prf-visualrois/V2v")

In [ ]:
import shutil
means = []
for _ in range(10):
    fx = FeatureExtractor(model='AlexNet', netset='Standard', device='cpu', pretrained=False)
    fx.extract(data_path=stimuli_path, save_path='AlexNet_Feat', consolidate_per_layer=False, layers_to_extract=["features.10"])
    results_dataframe = Ridge_Encoding(
        'AlexNet_Feat',
        roi_path,
        "AlexNet",
        n_folds=3,
        trn_tst_split=0.8,
        n_components=100,
        batch_size=64,
        return_correlations=True,
        save_path="results",
        alpha=1.0,
    )

    layer = results_dataframe.Layer.unique()[-1]
    last_layer_df = results_dataframe[results_dataframe.Layer == layer]
    print(last_layer_df.R.mean())
    means.append(last_layer_df.R.mean())
    shutil.rmtree('AlexNet_Feat')

In [ ]:
import numpy as np
print(np.mean(means))

In [ ]:
results_dataframe

In [ ]:
plotter = Plotting(results_dataframe)
plotter.plot_all_layers()

In [ ]:
import numpy as np

subject = 'subj07'

# Load the NPY file
lh = np.load(roi_path + f'/{subject}_roi-V2v_lh.npy')
rh = np.load(roi_path + f'/{subject}_roi-V2v_rh.npy')

# Concatenate the left and right hemisphere
roi = np.concatenate((lh, rh), axis=1)
# save as NPY file
np.save(roi_path + f'/{subject}_roi-V2v.npy', roi)